In [180]:
import numpy as np
import pandas as pd
import math
from imblearn.over_sampling import RandomOverSampler
import pymorphy2, re

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.layers import Input, Embedding, Activation, Flatten, Dense, concatenate
from keras.layers import Conv1D, MaxPooling1D, Dropout, LSTM
from keras.models import Model

from nltk.corpus import stopwords

In [226]:
#dataset
PATH_X_TRAIN=r'C:\Users\Annie\all_scripts\types_of_facts\sentences_train.csv'
PATH_Y_TRAIN=r'C:\Users\Annie\all_scripts\types_of_facts\types_train.csv'
PATH_X_VAL=r'C:\Users\Annie\all_scripts\types_of_facts\sentences_val.csv'
PATH_Y_VAL=r'C:\Users\Annie\all_scripts\types_of_facts\types_val.csv'
PATH_X_TEST=r'C:\Users\Annie\all_scripts\types_of_facts\sentences_test.csv'
PATH_Y_TEST=r'C:\Users\Annie\all_scripts\types_of_facts\types_test.csv'

POST_X_TRAIN=r'C:\Users\Annie\all_scripts\types_of_facts\post_train.csv'
POST_X_VAL=r'C:\Users\Annie\all_scripts\types_of_facts\post_val.csv'
POST_X_TEST=r'C:\Users\Annie\all_scripts\types_of_facts\post_test.csv'

PRED_X_TRAIN=r'C:\Users\Annie\all_scripts\types_of_facts\pred_train.csv'
PRED_X_VAL=r'C:\Users\Annie\all_scripts\types_of_facts\pred_val.csv'
PRED_X_TEST=r'C:\Users\Annie\all_scripts\types_of_facts\pred_test.csv'

POST2_X_TRAIN=r'C:\Users\Annie\all_scripts\types_of_facts\postpost_train.csv'
POST2_X_VAL=r'C:\Users\Annie\all_scripts\types_of_facts\postpost_val.csv'
POST2_X_TEST=r'C:\Users\Annie\all_scripts\types_of_facts\postpost_test.csv'

PRED2_X_TRAIN=r'C:\Users\Annie\all_scripts\types_of_facts\predpred_train.csv'
PRED2_X_VAL=r'C:\Users\Annie\all_scripts\types_of_facts\predpred_val.csv'
PRED2_X_TEST=r'C:\Users\Annie\all_scripts\types_of_facts\predpred_test.csv'

POST3_X_TRAIN=r'C:\Users\Annie\all_scripts\types_of_facts\postpostpost_train.csv'
POST3_X_VAL=r'C:\Users\Annie\all_scripts\types_of_facts\postpostpost_val.csv'
POST3_X_TEST=r'C:\Users\Annie\all_scripts\types_of_facts\postpostpost_test.csv'

PRED3_X_TRAIN=r'C:\Users\Annie\all_scripts\types_of_facts\predpredpred_train.csv'
PRED3_X_VAL=r'C:\Users\Annie\all_scripts\types_of_facts\predpredpred_val.csv'
PRED3_X_TEST=r'C:\Users\Annie\all_scripts\types_of_facts\predpredpred_test.csv'

POST4_X_TRAIN=r'C:\Users\Annie\all_scripts\types_of_facts\post4_train.csv'
POST4_X_VAL=r'C:\Users\Annie\all_scripts\types_of_facts\post4_val.csv'
POST4_X_TEST=r'C:\Users\Annie\all_scripts\types_of_facts\post4_test.csv'

PRED4_X_TRAIN=r'C:\Users\Annie\all_scripts\types_of_facts\pred4_train.csv'
PRED4_X_VAL=r'C:\Users\Annie\all_scripts\types_of_facts\pred4_val.csv'
PRED4_X_TEST=r'C:\Users\Annie\all_scripts\types_of_facts\pred4_test.csv'

In [227]:
x_train=pd.read_csv(PATH_X_TRAIN, sep=';', index_col=0)
y_train=pd.read_csv(PATH_Y_TRAIN, sep=';', index_col=0)
x_val=pd.read_csv(PATH_X_VAL, sep=';', index_col=0)
y_val=pd.read_csv(PATH_Y_VAL, sep=';', index_col=0)
x_test=pd.read_csv(PATH_X_TEST, sep=';', index_col=0)
y_test=pd.read_csv(PATH_Y_TEST, sep=';', index_col=0)

pred_train=pd.read_csv(PRED_X_TRAIN, sep=';', index_col=0)
pred_val=pd.read_csv(PRED_X_VAL, sep=';', index_col=0)
pred_test=pd.read_csv(PRED_X_TEST, sep=';', index_col=0)

post_train=pd.read_csv(POST_X_TRAIN, sep=';', index_col=0)
post_val=pd.read_csv(POST_X_VAL, sep=';', index_col=0)
post_test=pd.read_csv(POST_X_TEST, sep=';', index_col=0)

pred2_train=pd.read_csv(PRED2_X_TRAIN, sep=';', index_col=0)
pred2_val=pd.read_csv(PRED2_X_VAL, sep=';', index_col=0)
pred2_test=pd.read_csv(PRED2_X_TEST, sep=';', index_col=0)

post2_train=pd.read_csv(POST2_X_TRAIN, sep=';', index_col=0)
post2_val=pd.read_csv(POST2_X_VAL, sep=';', index_col=0)
post2_test=pd.read_csv(POST2_X_TEST, sep=';', index_col=0)

pred3_train=pd.read_csv(PRED3_X_TRAIN, sep=';', index_col=0)
pred3_val=pd.read_csv(PRED3_X_VAL, sep=';', index_col=0)
pred3_test=pd.read_csv(PRED3_X_TEST, sep=';', index_col=0)

post3_train=pd.read_csv(POST3_X_TRAIN, sep=';', index_col=0)
post3_val=pd.read_csv(POST3_X_VAL, sep=';', index_col=0)
post3_test=pd.read_csv(POST3_X_TEST, sep=';', index_col=0)

pred4_train=pd.read_csv(PRED4_X_TRAIN, sep=';', index_col=0)
pred4_val=pd.read_csv(PRED4_X_VAL, sep=';', index_col=0)
pred4_test=pd.read_csv(PRED4_X_TEST, sep=';', index_col=0)

post4_train=pd.read_csv(POST4_X_TRAIN, sep=';', index_col=0)
post4_val=pd.read_csv(POST4_X_VAL, sep=';', index_col=0)
post4_test=pd.read_csv(POST4_X_TEST, sep=';', index_col=0)

In [228]:
texts_train=x_train.values
texts_val=x_val.values
texts_test=x_test.values
classes_train=y_train.values
classes_val=y_val.values
classes_test=y_test.values

pred_train=pred_train.values
pred_val=pred_val.values
pred_test=pred_test.values

post_train=post_train.values
post_val=post_val.values
post_test=post_test.values

pred2_train=pred2_train.values
pred2_val=pred2_val.values
pred2_test=pred2_test.values

post2_train=post2_train.values
post2_val=post2_val.values
post2_test=post2_test.values

pred3_train=pred3_train.values
pred3_val=pred3_val.values
pred3_test=pred3_test.values

post3_train=post3_train.values
post3_val=post3_val.values
post3_test=post3_test.values

pred4_train=pred4_train.values
pred4_val=pred4_val.values
pred4_test=pred4_test.values

post4_train=post4_train.values
post4_val=post4_val.values
post4_test=post4_test.values

In [229]:
pred3_train.shape

(2315, 1)

In [230]:
texts_train.shape

(2315, 1)

In [231]:
nm=np.hstack((np.hstack((pred2_train, pred_train)),texts_train))
nm=np.hstack((np.hstack((nm, post_train)),post2_train))
nm=np.hstack((np.hstack((pred3_train, nm)),post3_train))
nm=np.hstack((np.hstack((pred4_train, nm)),post4_train))
nm.shape

(2315, 9)

In [232]:
ros = RandomOverSampler(random_state=42)
#texts_train, classes_train = ros.fit_resample(texts_train, classes_train)
nm, classes_train = ros.fit_resample(nm, classes_train)
classes_train=np.array(classes_train)

C:\Users\Annie\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [233]:
nm.shape

(8530, 9)

In [234]:
pred4_train, pred3_train, pred2_train, pred_train, texts_train, post_train, post2_train, post3_train, post4_train=np.hsplit(nm, (1, 2, 3, 4, 5, 6, 7, 8))

In [72]:
from gensim.models.keyedvectors import KeyedVectors
from tqdm import tqdm

print("считываю кутузовский файл")
path = r"C:\Users\Annie\Downloads\ruwikiruscorpora_upos_skipgram_300_2_2018.vec\ruwikiruscorpora_upos_skipgram_300_2_2018.vec"
w2v = KeyedVectors.load_word2vec_format(path, binary=False)
print("считал")

считываю кутузовский файл
считал


In [121]:
import os
import pymorphy2
import re
import pandas as pd
import keras
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

ma = pymorphy2.MorphAnalyzer()

def check_in_kutuzov(word):
    part_of_speech = str(ma.parse(word)[0].tag.POS).upper()
    if part_of_speech == 'ADJF':
        part_of_speech = "ADJ"
    elif part_of_speech == 'ADJS':
        part_of_speech = "ADJ"
    elif part_of_speech == 'NUMR':
        part_of_speech = "NUMR"
    elif part_of_speech == 'ADVB':
        part_of_speech == 'ADV'
    elif part_of_speech == 'INFN' or part_of_speech == 'PRTF' or part_of_speech == 'PRTS':
        part_of_speech = 'VERB'

    new_word = word + "_" + part_of_speech
    if new_word in w2v.vocab:
        return new_word
    else:
        return ""

def clean_text(text):
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ")
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) #deleting newlines and line-breaks
    text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', text) #deleting symbols 
    text = " ".join(ma.parse(word)[0].normal_form for word in text.split())
    text = ' '.join(word for word in text.split() if len(word)>3)
    all_words = text.split(' ')
    all_correct_words = ""
    for word in all_words:
        new_word = check_in_kutuzov(word)
        #print(new_word)
        if new_word !="":
            if all_correct_words=="":
                all_correct_words += new_word
            else:
                all_correct_words += " "+new_word
    #text = text.encode("utf-8")
    return all_correct_words

In [235]:
print('train')
for i in range(len(texts_train)):
    texts_train[i]=clean_text(texts_train[i][0])
print('val')
for i in range(len(texts_val)):
    texts_val[i]=clean_text(texts_val[i][0])
print('test')
for i in range(len(texts_test)):
    texts_test[i]=clean_text(texts_test[i][0])

print('post_train')
for i in range(len(post_train)):
    post_train[i]=clean_text(post_train[i][0])
print('post_val')
for i in range(len(post_val)):
    post_val[i]=clean_text(post_val[i][0])
print('post_test')
for i in range(len(post_test)):
    post_test[i]=clean_text(post_test[i][0])
    
print('pred_train')
for i in range(len(pred_train)):
    pred_train[i]=clean_text(pred_train[i][0])
print('pred_val')
for i in range(len(pred_val)):
    pred_val[i]=clean_text(pred_val[i][0])
print('pred_test')
for i in range(len(pred_test)):
    pred_test[i]=clean_text(pred_test[i][0])

train
val
test
post_train
post_val
post_test
pred_train
pred_val
pred_test


In [236]:
print('post2_train')
for i in range(len(post2_train)):
    post2_train[i]=clean_text(post2_train[i][0])
print('post2_val')
for i in range(len(post2_val)):
    post2_val[i]=clean_text(post2_val[i][0])
print('post2_test')
for i in range(len(post2_test)):
    post2_test[i]=clean_text(post2_test[i][0])
    
print('pred2_train')
for i in range(len(pred2_train)):
    pred2_train[i]=clean_text(pred2_train[i][0])
print('pred2_val')
for i in range(len(pred2_val)):
    pred2_val[i]=clean_text(pred2_val[i][0])
print('pred2_test')
for i in range(len(pred2_test)):
    pred2_test[i]=clean_text(pred2_test[i][0])
    
print('post3_train')
for i in range(len(post3_train)):
    post3_train[i]=clean_text(post3_train[i][0])
print('post3_val')
for i in range(len(post3_val)):
    post3_val[i]=clean_text(post3_val[i][0])
print('post3_test')
for i in range(len(post3_test)):
    post3_test[i]=clean_text(post3_test[i][0])
    
print('pred3_train')
for i in range(len(pred3_train)):
    pred3_train[i]=clean_text(pred3_train[i][0])
print('pred3_val')
for i in range(len(pred3_val)):
    pred3_val[i]=clean_text(pred3_val[i][0])
print('pred3_test')
for i in range(len(pred3_test)):
    pred3_test[i]=clean_text(pred3_test[i][0])
    
print('post4_train')
for i in range(len(post4_train)):
    post4_train[i]=clean_text(post4_train[i][0])
print('post4_val')
for i in range(len(post4_val)):
    post4_val[i]=clean_text(post4_val[i][0])
print('post4_test')
for i in range(len(post4_test)):
    post4_test[i]=clean_text(post4_test[i][0])
    
print('pred4_train')
for i in range(len(pred4_train)):
    pred4_train[i]=clean_text(pred4_train[i][0])
print('pred4_val')
for i in range(len(pred4_val)):
    pred4_val[i]=clean_text(pred4_val[i][0])
print('pred4_test')
for i in range(len(pred4_test)):
    pred4_test[i]=clean_text(pred4_test[i][0])

post2_train
post2_val
post2_test
pred2_train
pred2_val
pred2_test
post3_train
post3_val
post3_test
pred3_train
pred3_val
pred3_test
post4_train
post4_val
post4_test
pred4_train
pred4_val
pred4_test


In [237]:
texts_train[0]

array(['родиться_VERB февраль_NOUN март_NOUN юзовка_NOUN донецк_NOUN украина_NOUN'],
      dtype=object)

In [238]:
x_train=[]
for t in texts_train:
    x_train.append(t[0])
x_val=[]
for t in texts_val:
    x_val.append(t[0])
x_test=[]
for t in texts_test:
    x_test.append(t[0])
    
pr_train=[]
for t in pred_train:
    pr_train.append(t[0])
pr_val=[]
for t in pred_val:
    pr_val.append(t[0])
pr_test=[]
for t in pred_test:
    pr_test.append(t[0])
    
po_train=[]
for t in post_train:
    po_train.append(t[0])
po_val=[]
for t in post_val:
    po_val.append(t[0])
po_test=[]
for t in post_test:
    po_test.append(t[0])
    
pr2_train=[]
for t in pred2_train:
    pr2_train.append(t[0])
pr2_val=[]
for t in pred2_val:
    pr2_val.append(t[0])
pr2_test=[]
for t in pred2_test:
    pr2_test.append(t[0])
    
po2_train=[]
for t in post2_train:
    po2_train.append(t[0])
po2_val=[]
for t in post2_val:
    po2_val.append(t[0])
po2_test=[]
for t in post2_test:
    po2_test.append(t[0])
    
pr3_train=[]
for t in pred3_train:
    pr3_train.append(t[0])
pr3_val=[]
for t in pred3_val:
    pr3_val.append(t[0])
pr3_test=[]
for t in pred3_test:
    pr3_test.append(t[0])
    
po3_train=[]
for t in post3_train:
    po3_train.append(t[0])
po3_val=[]
for t in post3_val:
    po3_val.append(t[0])
po3_test=[]
for t in post3_test:
    po3_test.append(t[0])
    
pr4_train=[]
for t in pred4_train:
    pr4_train.append(t[0])
pr4_val=[]
for t in pred4_val:
    pr4_val.append(t[0])
pr4_test=[]
for t in pred4_test:
    pr4_test.append(t[0])
    
po4_train=[]
for t in post4_train:
    po4_train.append(t[0])
po4_val=[]
for t in post4_val:
    po4_val.append(t[0])
po4_test=[]
for t in post4_test:
    po4_test.append(t[0])

In [239]:
x_train[0]

'родиться_VERB февраль_NOUN март_NOUN юзовка_NOUN донецк_NOUN украина_NOUN'

In [240]:
#classes preprocessing

#dictionary for languages
classes = np.unique(np.array(classes_train))
nums=np.arange(len(classes))
d = dict(zip(classes,nums))

classes_train=[d[c] for c in classes_train] 
classes_val=[d[c[0]] for c in list(classes_val)]
classes_test=[d[c[0]] for c in list(classes_test)] 
classes_train=to_categorical(classes_train)
classes_val=to_categorical(classes_val)
classes_test=to_categorical(classes_test)

In [241]:
num_classes = len(classes)

# Максимальное количество слов в самом длинном тексте
max_words = 0

for desc in x_train:
#for desc in descriptions:
    words = len(desc.split())
    if words > max_words:
        max_words = words
print('Максимальное количество слов в самом длинном тексте: {} слов'.format(max_words))

maxSequenceLength = max_words

Максимальное количество слов в самом длинном тексте: 58 слов


In [242]:
# define class labels
# prepare tokenizer
t = Tokenizer()

for i in range(len(x_train)):
    x_train[i] = x_train[i].replace('_','')
for i in range(len(x_val)):
    x_val[i] = x_val[i].replace('_','')
for i in range(len(x_test)):
    x_test[i] = x_test[i].replace('_','')

for i in range(len(po_train)):
    po_train[i] = po_train[i].replace('_','')
for i in range(len(po_val)):
    po_val[i] = po_val[i].replace('_','')
for i in range(len(po_test)):
    po_test[i] = po_test[i].replace('_','')

for i in range(len(pr_train)):
    pr_train[i] = pr_train[i].replace('_','')
for i in range(len(pr_val)):
    pr_val[i] = pr_val[i].replace('_','')
for i in range(len(pr_test)):
    pr_test[i] = pr_test[i].replace('_','')
    
for i in range(len(po2_train)):
    po2_train[i] = po2_train[i].replace('_','')
for i in range(len(po2_val)):
    po2_val[i] = po2_val[i].replace('_','')
for i in range(len(po2_test)):
    po2_test[i] = po2_test[i].replace('_','')

for i in range(len(pr2_train)):
    pr2_train[i] = pr2_train[i].replace('_','')
for i in range(len(pr2_val)):
    pr2_val[i] = pr2_val[i].replace('_','')
for i in range(len(pr2_test)):
    pr2_test[i] = pr2_test[i].replace('_','')
    
for i in range(len(po3_train)):
    po3_train[i] = po3_train[i].replace('_','')
for i in range(len(po3_val)):
    po3_val[i] = po3_val[i].replace('_','')
for i in range(len(po3_test)):
    po3_test[i] = po3_test[i].replace('_','')

for i in range(len(pr3_train)):
    pr3_train[i] = pr3_train[i].replace('_','')
for i in range(len(pr3_val)):
    pr3_val[i] = pr3_val[i].replace('_','')
for i in range(len(pr3_test)):
    pr3_test[i] = pr3_test[i].replace('_','')
    
for i in range(len(po4_train)):
    po4_train[i] = po4_train[i].replace('_','')
for i in range(len(po4_val)):
    po4_val[i] = po4_val[i].replace('_','')
for i in range(len(po4_test)):
    po4_test[i] = po4_test[i].replace('_','')

for i in range(len(pr4_train)):
    pr4_train[i] = pr4_train[i].replace('_','')
for i in range(len(pr4_val)):
    pr4_val[i] = pr4_val[i].replace('_','')
for i in range(len(pr4_test)):
    pr4_test[i] = pr4_test[i].replace('_','')

In [243]:
#t.fit_on_texts(x_train)
t.fit_on_texts(x_train)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs_train = t.texts_to_sequences(x_train)
encoded_docs_val = t.texts_to_sequences(x_val)
encoded_docs_test = t.texts_to_sequences(x_test)

encoded_docs_pr_train = t.texts_to_sequences(pr_train)
encoded_docs_pr_val = t.texts_to_sequences(pr_val)
encoded_docs_pr_test = t.texts_to_sequences(pr_test)

encoded_docs_po_train = t.texts_to_sequences(po_train)
encoded_docs_po_val = t.texts_to_sequences(po_val)
encoded_docs_po_test = t.texts_to_sequences(po_test)

encoded_docs_pr2_train = t.texts_to_sequences(pr2_train)
encoded_docs_pr2_val = t.texts_to_sequences(pr2_val)
encoded_docs_pr2_test = t.texts_to_sequences(pr2_test)

encoded_docs_po2_train = t.texts_to_sequences(po2_train)
encoded_docs_po2_val = t.texts_to_sequences(po2_val)
encoded_docs_po2_test = t.texts_to_sequences(po2_test)

encoded_docs_pr3_train = t.texts_to_sequences(pr3_train)
encoded_docs_pr3_val = t.texts_to_sequences(pr3_val)
encoded_docs_pr3_test = t.texts_to_sequences(pr3_test)

encoded_docs_po3_train = t.texts_to_sequences(po3_train)
encoded_docs_po3_val = t.texts_to_sequences(po3_val)
encoded_docs_po3_test = t.texts_to_sequences(po3_test)

encoded_docs_pr4_train = t.texts_to_sequences(pr4_train)
encoded_docs_pr4_val = t.texts_to_sequences(pr4_val)
encoded_docs_pr4_test = t.texts_to_sequences(pr4_test)

encoded_docs_po4_train = t.texts_to_sequences(po4_train)
encoded_docs_po4_val = t.texts_to_sequences(po4_val)
encoded_docs_po4_test = t.texts_to_sequences(po4_test)

# pad documents to a max length of 4 words

padded_docs_train = sequence.pad_sequences(encoded_docs_train, maxlen=maxSequenceLength)
padded_docs_val = sequence.pad_sequences(encoded_docs_val, maxlen=maxSequenceLength)
padded_docs_test = sequence.pad_sequences(encoded_docs_test, maxlen=maxSequenceLength)

padded_docs_pr_train = sequence.pad_sequences(encoded_docs_pr_train, maxlen=maxSequenceLength)
padded_docs_pr_val = sequence.pad_sequences(encoded_docs_pr_val, maxlen=maxSequenceLength)
padded_docs_pr_test = sequence.pad_sequences(encoded_docs_pr_test, maxlen=maxSequenceLength)

padded_docs_po_train = sequence.pad_sequences(encoded_docs_po_train, maxlen=maxSequenceLength)
padded_docs_po_val = sequence.pad_sequences(encoded_docs_po_val, maxlen=maxSequenceLength)
padded_docs_po_test = sequence.pad_sequences(encoded_docs_po_test, maxlen=maxSequenceLength)

padded_docs_pr2_train = sequence.pad_sequences(encoded_docs_pr2_train, maxlen=maxSequenceLength)
padded_docs_pr2_val = sequence.pad_sequences(encoded_docs_pr2_val, maxlen=maxSequenceLength)
padded_docs_pr2_test = sequence.pad_sequences(encoded_docs_pr2_test, maxlen=maxSequenceLength)

padded_docs_po2_train = sequence.pad_sequences(encoded_docs_po2_train, maxlen=maxSequenceLength)
padded_docs_po2_val = sequence.pad_sequences(encoded_docs_po2_val, maxlen=maxSequenceLength)
padded_docs_po2_test = sequence.pad_sequences(encoded_docs_po2_test, maxlen=maxSequenceLength)

padded_docs_pr3_train = sequence.pad_sequences(encoded_docs_pr3_train, maxlen=maxSequenceLength)
padded_docs_pr3_val = sequence.pad_sequences(encoded_docs_pr3_val, maxlen=maxSequenceLength)
padded_docs_pr3_test = sequence.pad_sequences(encoded_docs_pr3_test, maxlen=maxSequenceLength)

padded_docs_po3_train = sequence.pad_sequences(encoded_docs_po3_train, maxlen=maxSequenceLength)
padded_docs_po3_val = sequence.pad_sequences(encoded_docs_po3_val, maxlen=maxSequenceLength)
padded_docs_po3_test = sequence.pad_sequences(encoded_docs_po3_test, maxlen=maxSequenceLength)

padded_docs_pr4_train = sequence.pad_sequences(encoded_docs_pr4_train, maxlen=maxSequenceLength)
padded_docs_pr4_val = sequence.pad_sequences(encoded_docs_pr4_val, maxlen=maxSequenceLength)
padded_docs_pr4_test = sequence.pad_sequences(encoded_docs_pr4_test, maxlen=maxSequenceLength)

padded_docs_po4_train = sequence.pad_sequences(encoded_docs_po4_train, maxlen=maxSequenceLength)
padded_docs_po4_val = sequence.pad_sequences(encoded_docs_po4_val, maxlen=maxSequenceLength)
padded_docs_po4_test = sequence.pad_sequences(encoded_docs_po4_test, maxlen=maxSequenceLength)

total_unique_words = len(t.word_counts)
print('Всего уникальных слов в словаре: {}'.format(total_unique_words))

Всего уникальных слов в словаре: 4531


In [244]:
t.word_index.items()

dict_items([('родитьсяverb', 1), ('членnoun', 2), ('семьяnoun', 3), ('москваnoun', 4), ('адамсnoun', 5), ('городnoun', 6), ('университетnoun', 7), ('окончитьverb', 8), ('умеретьverb', 9), ('областьnoun', 10), ('московскийadj', 11), ('институтnoun', 12), ('житьverb', 13), ('ребёнокnoun', 14), ('работатьverb', 15), ('наукаnoun', 16), ('российскийadj', 17), ('времяnoun', 18), ('академияnoun', 19), ('школаnoun', 20), ('кладбищеnoun', 21), ('статьverb', 22), ('скончатьсяverb', 23), ('советnoun', 24), ('октябрьnoun', 25), ('сентябрьnoun', 26), ('сссрnoun', 27), ('александрnoun', 28), ('государственныйadj', 29), ('метрnoun', 30), ('работаnoun', 31), ('вернутьсяverb', 32), ('похоронныйadj', 33), ('отецnoun', 34), ('иметьverb', 35), ('мартnoun', 36), ('получитьverb', 37), ('переехатьverb', 38), ('научныйadj', 39), ('войнаnoun', 40), ('обществоnoun', 41), ('декабрьnoun', 42), ('районnoun', 43), ('апрельnoun', 44), ('корреспондентnoun', 45), ('выйтиverb', 46), ('матьnoun', 47), ('диссертацияnoun'

In [245]:
import numpy as np
#ДОЛГОЕ ПОСТРОЕНИЕ МАТРИЦЫ ВЕСОВ
# create a weight matrix for words in training docs
print("Создаю матрицу")
embedding_matrix = np.zeros((vocab_size, 300))
print("Начинаем")
for word, i in t.word_index.items():
    
    try:
        new_word = word.replace('adj', '_ADJ')
        new_word = new_word.replace('noun', '_NOUN')
        new_word = new_word.replace('verb', '_VERB')
        embedding_vector = w2v[new_word]

        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except:
        print(word)
        print(i) 

Создаю матрицу
Начинаем


In [247]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, concatenate
from keras.optimizers import RMSprop
from keras.layers import Bidirectional
'''
#без контекста

inputs=Input(shape=(maxSequenceLength,), name='input', dtype='int64')
x=Embedding(vocab_size, 300, name='word2vec', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs)

x=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64')(x)

x=Dense(32, activation='tanh', name='fully_connected_32')(x)
predictions=Dense(num_classes, activation='softmax', name='output')(x)
model=Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
'''
'''
#1:0
inputs_x=Input(shape=(maxSequenceLength,), name='input_pred', dtype='int64')
inputs_y=Input(shape=(maxSequenceLength,), name='input', dtype='int64')
inputs_z=Input(shape=(maxSequenceLength,), name='input_post', dtype='int64')

x=Embedding(vocab_size, 300, name='word2vec_pred', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x)
y=Embedding(vocab_size, 300, name='word2vec', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_y)
z=Embedding(vocab_size, 300, name='word2vec_post', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z)

x=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred')(x)
y=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64')(y)
z=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post')(z)

x=Model(inputs=inputs_x, outputs=x)
y=Model(inputs=inputs_y, outputs=y)
z=Model(inputs=inputs_z, outputs=z)

combined=concatenate([x.output, y.output, z.output], name='concat')

ex=Dense(32, activation='tanh', name='fully_connected_32')(combined)
predictions=Dense(num_classes, activation='softmax', name='output')(ex)
model=Model(inputs=[inputs_x, inputs_y, inputs_z], outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
'''
'''
#0:1
inputs_x=Input(shape=(maxSequenceLength,), name='input_pred', dtype='int64')
inputs_y=Input(shape=(maxSequenceLength,), name='input', dtype='int64')
inputs_z=Input(shape=(maxSequenceLength,), name='input_post', dtype='int64')

x=Embedding(vocab_size, 300, name='word2vec_pred', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x)
y=Embedding(vocab_size, 300, name='word2vec', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_y)
z=Embedding(vocab_size, 300, name='word2vec_post', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z)

x=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred')(x)
y=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64')(y)
z=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post')(z)

x=Dense(32, activation='tanh', name='fully_connected_32_pred')(x)
y=Dense(32, activation='tanh', name='fully_connected_32')(y)
z=Dense(32, activation='tanh', name='fully_connected_32_post')(z)

x=Model(inputs=inputs_x, outputs=x)
y=Model(inputs=inputs_y, outputs=y)
z=Model(inputs=inputs_z, outputs=z)

combined=concatenate([x.output, y.output, z.output], name='concat')

predictions=Dense(num_classes, activation='softmax', name='output')(combined)
model=Model(inputs=[inputs_x, inputs_y, inputs_z], outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


'''
'''
#1:1
inputs_x=Input(shape=(maxSequenceLength,), name='input_pred', dtype='int64')
inputs_y=Input(shape=(maxSequenceLength,), name='input', dtype='int64')
inputs_z=Input(shape=(maxSequenceLength,), name='input_post', dtype='int64')

x=Embedding(vocab_size, 300, name='word2vec_pred', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x)
y=Embedding(vocab_size, 300, name='word2vec', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_y)
z=Embedding(vocab_size, 300, name='word2vec_post', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z)

x=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred')(x)
y=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64')(y)
z=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post')(z)

x_m=Model(inputs=inputs_x, outputs=x)
y_m=Model(inputs=inputs_y, outputs=y)
z_m=Model(inputs=inputs_z, outputs=z)

combined=concatenate([x_m.output, y_m.output, z_m.output], name='concat_first')

x=Dense(32, activation='tanh', name='fully_connected_32_pred')(x)
ex=Dense(32, activation='tanh', name='fully_connected_32')(combined)
z=Dense(32, activation='tanh', name='fully_connected_32_post')(z)

x_m1=Model(inputs=inputs_x, outputs=x)
y_m1=Model(inputs=[inputs_x, inputs_y, inputs_z], outputs=ex)
z_m1=Model(inputs=inputs_z, outputs=z)

combined1=concatenate([x_m1.output, y_m1.output, z_m1.output], name='concat_second')

predictions=Dense(num_classes, activation='softmax', name='output')(combined1)
model=Model(inputs=[inputs_x, inputs_y, inputs_z], outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
'''
'''
#2:0
inputs_x1=Input(shape=(maxSequenceLength,), name='input_pred2', dtype='int64')
inputs_x=Input(shape=(maxSequenceLength,), name='input_pred1', dtype='int64')
inputs_y=Input(shape=(maxSequenceLength,), name='input', dtype='int64')
inputs_z=Input(shape=(maxSequenceLength,), name='input_post1', dtype='int64')
inputs_z1=Input(shape=(maxSequenceLength,), name='input_post2', dtype='int64')

x1=Embedding(vocab_size, 300, name='word2vec_pred2', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x1)
x=Embedding(vocab_size, 300, name='word2vec_pred1', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x)
y=Embedding(vocab_size, 300, name='word2vec', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_y)
z=Embedding(vocab_size, 300, name='word2vec_post1', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z)
z1=Embedding(vocab_size, 300, name='word2vec_post2', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z1)

x1=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred2')(x1)
x=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred1')(x)
y=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64')(y)
z=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post1')(z)
z1=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post2')(z1)

x1=Model(inputs=inputs_x1, outputs=x1)
x=Model(inputs=inputs_x, outputs=x)
y=Model(inputs=inputs_y, outputs=y)
z=Model(inputs=inputs_z, outputs=z)
z1=Model(inputs=inputs_z1, outputs=z1)

combined=concatenate([x1.output, x.output, y.output, z.output, z1.output], name='concat')

ex=Dense(32, activation='tanh', name='fully_connected_32')(combined)
predictions=Dense(num_classes, activation='softmax', name='output')(ex)
model=Model(inputs=[inputs_x1, inputs_x, inputs_y, inputs_z, inputs_z1], outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
'''
'''
#0:2
inputs_x1=Input(shape=(maxSequenceLength,), name='input_pred2', dtype='int64')
inputs_x=Input(shape=(maxSequenceLength,), name='input_pred1', dtype='int64')
inputs_y=Input(shape=(maxSequenceLength,), name='input', dtype='int64')
inputs_z=Input(shape=(maxSequenceLength,), name='input_post1', dtype='int64')
inputs_z1=Input(shape=(maxSequenceLength,), name='input_post2', dtype='int64')

x1=Embedding(vocab_size, 300, name='word2vec_pred2', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x1)
x=Embedding(vocab_size, 300, name='word2vec_pred1', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x)
y=Embedding(vocab_size, 300, name='word2vec', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_y)
z=Embedding(vocab_size, 300, name='word2vec_post1', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z)
z1=Embedding(vocab_size, 300, name='word2vec_post2', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z1)

x1=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred2')(x1)
x=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred1')(x)
y=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64')(y)
z=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post1')(z)
z1=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post2')(z1)

x1=Dense(32, activation='tanh', name='fully_connected_32_pred2')(x1)
x=Dense(32, activation='tanh', name='fully_connected_32_pred1')(x)
y=Dense(32, activation='tanh', name='fully_connected_32')(y)
z=Dense(32, activation='tanh', name='fully_connected_32_post1')(z)
z1=Dense(32, activation='tanh', name='fully_connected_32_post2')(z1)

x1=Model(inputs=inputs_x1, outputs=x1)
x=Model(inputs=inputs_x, outputs=x)
y=Model(inputs=inputs_y, outputs=y)
z=Model(inputs=inputs_z, outputs=z)
z1=Model(inputs=inputs_z1, outputs=z1)

combined=concatenate([x1.output, x.output, y.output, z.output, z1.output], name='concat')

predictions=Dense(num_classes, activation='softmax', name='output')(combined)
model=Model(inputs=[inputs_x1, inputs_x, inputs_y, inputs_z, inputs_z1], outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
'''
'''
#2:2
inputs_x1=Input(shape=(maxSequenceLength,), name='input_pred2', dtype='int64')
inputs_x=Input(shape=(maxSequenceLength,), name='input_pred1', dtype='int64')
inputs_y=Input(shape=(maxSequenceLength,), name='input', dtype='int64')
inputs_z=Input(shape=(maxSequenceLength,), name='input_post1', dtype='int64')
inputs_z1=Input(shape=(maxSequenceLength,), name='input_post2', dtype='int64')

x1=Embedding(vocab_size, 300, name='word2vec_pred2', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x1)
x=Embedding(vocab_size, 300, name='word2vec_pred1', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x)
y=Embedding(vocab_size, 300, name='word2vec', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_y)
z=Embedding(vocab_size, 300, name='word2vec_post1', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z)
z1=Embedding(vocab_size, 300, name='word2vec_post2', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z1)

x1=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred2')(x1)
x=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred1')(x)
y=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64')(y)
z=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post1')(z)
z1=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post2')(z1)

x_m_1=Model(inputs=inputs_x1, outputs=x1)
x_m=Model(inputs=inputs_x, outputs=x)
y_m=Model(inputs=inputs_y, outputs=y)
z_m=Model(inputs=inputs_z, outputs=z)
z_m_1=Model(inputs=inputs_z1, outputs=z1)

combined=concatenate([x_m_1.output, x_m.output, y_m.output, z_m.output, z_m_1.output], name='concat_first')

x1=Dense(32, activation='tanh', name='fully_connected_32_pred2')(x1)
x=Dense(32, activation='tanh', name='fully_connected_32_pred1')(x)
ex=Dense(32, activation='tanh', name='fully_connected_32')(combined)
z=Dense(32, activation='tanh', name='fully_connected_32_post1')(z)
z1=Dense(32, activation='tanh', name='fully_connected_32_post2')(z1)

x_m1_1=Model(inputs=inputs_x1, outputs=x1)
x_m1=Model(inputs=inputs_x, outputs=x)
y_m1=Model(inputs=[inputs_x1, inputs_x, inputs_y, inputs_z, inputs_z1], outputs=ex)
z_m1=Model(inputs=inputs_z, outputs=z)
z_m1_1=Model(inputs=inputs_z1, outputs=z1)

combined1=concatenate([x_m1_1.output, x_m1.output, y_m1.output, z_m1.output, z_m1_1.output], name='concat_second')

predictions=Dense(num_classes, activation='softmax', name='output')(combined1)
model=Model(inputs=[inputs_x1, inputs_x, inputs_y, inputs_z, inputs_z1], outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
'''
'''
#3:0
inputs_x2=Input(shape=(maxSequenceLength,), name='input_pred3', dtype='int64')
inputs_x1=Input(shape=(maxSequenceLength,), name='input_pred2', dtype='int64')
inputs_x=Input(shape=(maxSequenceLength,), name='input_pred1', dtype='int64')
inputs_y=Input(shape=(maxSequenceLength,), name='input', dtype='int64')
inputs_z=Input(shape=(maxSequenceLength,), name='input_post1', dtype='int64')
inputs_z1=Input(shape=(maxSequenceLength,), name='input_post2', dtype='int64')
inputs_z2=Input(shape=(maxSequenceLength,), name='input_post3', dtype='int64')

x2=Embedding(vocab_size, 300, name='word2vec_pred3', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x2)
x1=Embedding(vocab_size, 300, name='word2vec_pred2', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x1)
x=Embedding(vocab_size, 300, name='word2vec_pred1', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x)
y=Embedding(vocab_size, 300, name='word2vec', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_y)
z=Embedding(vocab_size, 300, name='word2vec_post1', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z)
z1=Embedding(vocab_size, 300, name='word2vec_post2', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z1)
z2=Embedding(vocab_size, 300, name='word2vec_post3', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z2)

x2=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred3')(x2)
x1=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred2')(x1)
x=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred1')(x)
y=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64')(y)
z=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post1')(z)
z1=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post2')(z1)
z2=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post3')(z2)

x2=Model(inputs=inputs_x2, outputs=x2)
x1=Model(inputs=inputs_x1, outputs=x1)
x=Model(inputs=inputs_x, outputs=x)
y=Model(inputs=inputs_y, outputs=y)
z=Model(inputs=inputs_z, outputs=z)
z1=Model(inputs=inputs_z1, outputs=z1)
z2=Model(inputs=inputs_z2, outputs=z2)

combined=concatenate([x2.output, x1.output, x.output, y.output, z.output, z1.output, z2.output], name='concat')

ex=Dense(32, activation='tanh', name='fully_connected_32')(combined)
predictions=Dense(num_classes, activation='softmax', name='output')(ex)
model=Model(inputs=[inputs_x2, inputs_x1, inputs_x, inputs_y, inputs_z, inputs_z1, inputs_z2], outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
'''
'''
#0:3
inputs_x2=Input(shape=(maxSequenceLength,), name='input_pred3', dtype='int64')
inputs_x1=Input(shape=(maxSequenceLength,), name='input_pred2', dtype='int64')
inputs_x=Input(shape=(maxSequenceLength,), name='input_pred1', dtype='int64')
inputs_y=Input(shape=(maxSequenceLength,), name='input', dtype='int64')
inputs_z=Input(shape=(maxSequenceLength,), name='input_post1', dtype='int64')
inputs_z1=Input(shape=(maxSequenceLength,), name='input_post2', dtype='int64')
inputs_z2=Input(shape=(maxSequenceLength,), name='input_post3', dtype='int64')

x2=Embedding(vocab_size, 300, name='word2vec_pred3', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x2)
x1=Embedding(vocab_size, 300, name='word2vec_pred2', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x1)
x=Embedding(vocab_size, 300, name='word2vec_pred1', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x)
y=Embedding(vocab_size, 300, name='word2vec', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_y)
z=Embedding(vocab_size, 300, name='word2vec_post1', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z)
z1=Embedding(vocab_size, 300, name='word2vec_post2', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z1)
z2=Embedding(vocab_size, 300, name='word2vec_post3', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z2)

x2=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred3')(x2)
x1=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred2')(x1)
x=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred1')(x)
y=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64')(y)
z=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post1')(z)
z1=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post2')(z1)
z2=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post3')(z2)

x2=Dense(32, activation='tanh', name='fully_connected_32_pred3')(x2)
x1=Dense(32, activation='tanh', name='fully_connected_32_pred2')(x1)
x=Dense(32, activation='tanh', name='fully_connected_32_pred1')(x)
y=Dense(32, activation='tanh', name='fully_connected_32')(y)
z=Dense(32, activation='tanh', name='fully_connected_32_post1')(z)
z1=Dense(32, activation='tanh', name='fully_connected_32_post2')(z1)
z2=Dense(32, activation='tanh', name='fully_connected_32_post3')(z2)

x2=Model(inputs=inputs_x2, outputs=x2)
x1=Model(inputs=inputs_x1, outputs=x1)
x=Model(inputs=inputs_x, outputs=x)
y=Model(inputs=inputs_y, outputs=y)
z=Model(inputs=inputs_z, outputs=z)
z1=Model(inputs=inputs_z1, outputs=z1)
z2=Model(inputs=inputs_z2, outputs=z2)

combined=concatenate([x2.output, x1.output, x.output, y.output, z.output, z1.output, z2.output], name='concat')

predictions=Dense(num_classes, activation='softmax', name='output')(combined)
model=Model(inputs=[inputs_x2, inputs_x1, inputs_x, inputs_y, inputs_z, inputs_z1, inputs_z2], outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
'''
'''
#3_3
inputs_x2=Input(shape=(maxSequenceLength,), name='input_pred3', dtype='int64')
inputs_x1=Input(shape=(maxSequenceLength,), name='input_pred2', dtype='int64')
inputs_x=Input(shape=(maxSequenceLength,), name='input_pred1', dtype='int64')
inputs_y=Input(shape=(maxSequenceLength,), name='input', dtype='int64')
inputs_z=Input(shape=(maxSequenceLength,), name='input_post1', dtype='int64')
inputs_z1=Input(shape=(maxSequenceLength,), name='input_post2', dtype='int64')
inputs_z2=Input(shape=(maxSequenceLength,), name='input_post3', dtype='int64')

x2=Embedding(vocab_size, 300, name='word2vec_pred3', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x2)
x1=Embedding(vocab_size, 300, name='word2vec_pred2', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x1)
x=Embedding(vocab_size, 300, name='word2vec_pred1', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x)
y=Embedding(vocab_size, 300, name='word2vec', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_y)
z=Embedding(vocab_size, 300, name='word2vec_post1', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z)
z1=Embedding(vocab_size, 300, name='word2vec_post2', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z1)
z2=Embedding(vocab_size, 300, name='word2vec_post3', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z2)

x2=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred3')(x2)
x1=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred2')(x1)
x=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred1')(x)
y=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64')(y)
z=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post1')(z)
z1=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post2')(z1)
z2=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post3')(z2)

x_m_2=Model(inputs=inputs_x2, outputs=x2)
x_m_1=Model(inputs=inputs_x1, outputs=x1)
x_m=Model(inputs=inputs_x, outputs=x)
y_m=Model(inputs=inputs_y, outputs=y)
z_m=Model(inputs=inputs_z, outputs=z)
z_m_1=Model(inputs=inputs_z1, outputs=z1)
z_m_2=Model(inputs=inputs_z2, outputs=z2)

combined=concatenate([x_m_2.output, x_m_1.output, x_m.output, y_m.output, z_m.output, z_m_1.output, z_m_2.output], name='concat_first')

x2=Dense(32, activation='tanh', name='fully_connected_32_pred3')(x2)
x1=Dense(32, activation='tanh', name='fully_connected_32_pred2')(x1)
x=Dense(32, activation='tanh', name='fully_connected_32_pred1')(x)
y=Dense(32, activation='tanh', name='fully_connected_32')(y)
z=Dense(32, activation='tanh', name='fully_connected_32_post1')(z)
z1=Dense(32, activation='tanh', name='fully_connected_32_post2')(z1)
z2=Dense(32, activation='tanh', name='fully_connected_32_post3')(z2)

x_m1_2=Model(inputs=inputs_x2, outputs=x2)
x_m1_1=Model(inputs=inputs_x1, outputs=x1)
x_m1=Model(inputs=inputs_x, outputs=x)
y_m1=Model(inputs=[inputs_x2, inputs_x1, inputs_x, inputs_y, inputs_z, inputs_z1, inputs_z2], outputs=y)
z_m1=Model(inputs=inputs_z, outputs=z)
z_m1_1=Model(inputs=inputs_z1, outputs=z1)
z_m1_2=Model(inputs=inputs_z2, outputs=z2)

combined1=concatenate([x_m1_2.output, x_m1_1.output, x_m1.output, y_m1.output, z_m1.output, z_m1_1.output, z_m1_2.output], name='concat_second')

predictions=Dense(num_classes, activation='softmax', name='output')(combined1)
model=Model(inputs=[inputs_x2, inputs_x1, inputs_x, inputs_y, inputs_z, inputs_z1, inputs_z2], outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
'''
#'''
#3:0
inputs_x3=Input(shape=(maxSequenceLength,), name='input_pred4', dtype='int64')
inputs_x2=Input(shape=(maxSequenceLength,), name='input_pred3', dtype='int64')
inputs_x1=Input(shape=(maxSequenceLength,), name='input_pred2', dtype='int64')
inputs_x=Input(shape=(maxSequenceLength,), name='input_pred1', dtype='int64')
inputs_y=Input(shape=(maxSequenceLength,), name='input', dtype='int64')
inputs_z=Input(shape=(maxSequenceLength,), name='input_post1', dtype='int64')
inputs_z1=Input(shape=(maxSequenceLength,), name='input_post2', dtype='int64')
inputs_z2=Input(shape=(maxSequenceLength,), name='input_post3', dtype='int64')
inputs_z3=Input(shape=(maxSequenceLength,), name='input_post4', dtype='int64')

x3=Embedding(vocab_size, 300, name='word2vec_pred4', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x3)
x2=Embedding(vocab_size, 300, name='word2vec_pred3', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x2)
x1=Embedding(vocab_size, 300, name='word2vec_pred2', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x1)
x=Embedding(vocab_size, 300, name='word2vec_pred1', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x)
y=Embedding(vocab_size, 300, name='word2vec', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_y)
z=Embedding(vocab_size, 300, name='word2vec_post1', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z)
z1=Embedding(vocab_size, 300, name='word2vec_post2', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z1)
z2=Embedding(vocab_size, 300, name='word2vec_post3', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z2)
z3=Embedding(vocab_size, 300, name='word2vec_post4', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_z3)

x3=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred4')(x3)
x2=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred3')(x2)
x1=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred2')(x1)
x=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_pred1')(x)
y=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64')(y)
z=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post1')(z)
z1=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post2')(z1)
z2=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post3')(z2)
z3=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='lstm_64_post4')(z3)

x3=Model(inputs=inputs_x3, outputs=x3)
x2=Model(inputs=inputs_x2, outputs=x2)
x1=Model(inputs=inputs_x1, outputs=x1)
x=Model(inputs=inputs_x, outputs=x)
y=Model(inputs=inputs_y, outputs=y)
z=Model(inputs=inputs_z, outputs=z)
z1=Model(inputs=inputs_z1, outputs=z1)
z2=Model(inputs=inputs_z2, outputs=z2)
z3=Model(inputs=inputs_z3, outputs=z3)

combined=concatenate([x3.output, x2.output, x1.output, x.output, y.output, z.output, z1.output, z2.output, z3.output], name='concat')

ex=Dense(32, activation='tanh', name='fully_connected_32')(combined)
predictions=Dense(num_classes, activation='softmax', name='output')(ex)
model=Model(inputs=[inputs_x3, inputs_x2, inputs_x1, inputs_x, inputs_y, inputs_z, inputs_z1, inputs_z2, inputs_z3], outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
#'''

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_pred4 (InputLayer)        (None, 58)           0                                            
__________________________________________________________________________________________________
input_pred3 (InputLayer)        (None, 58)           0                                            
__________________________________________________________________________________________________
input_pred2 (InputLayer)        (None, 58)           0                                            
__________________________________________________________________________________________________
input_pred1 (InputLayer)        (None, 58)           0                                            
__________________________________________________________________________________________________
input (Inp

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='0_3.png')

In [248]:
for i in range(20):
    print(i)
    history = model.fit([padded_docs_pr4_train, padded_docs_pr3_train, padded_docs_pr2_train, padded_docs_pr_train, padded_docs_train, padded_docs_po_train, padded_docs_po2_train, padded_docs_po3_train, padded_docs_po4_train], classes_train, epochs = 1, verbose=2, validation_data=([padded_docs_pr4_test, padded_docs_pr3_test, padded_docs_pr2_test, padded_docs_pr_test, padded_docs_test, padded_docs_po_test, padded_docs_po2_test, padded_docs_po3_test, padded_docs_po4_test], classes_test))
    predict = np.argmax(model.predict([padded_docs_pr4_val, padded_docs_pr3_val, padded_docs_pr2_val, padded_docs_pr_val, padded_docs_val, padded_docs_po_val, padded_docs_po2_val, padded_docs_po3_val, padded_docs_po4_val]), axis=1)
    answer = np.argmax(classes_val, axis=1)
    print('F1-score: %f' % (f1_score(predict, answer, average="macro")*100))

0
Train on 8530 samples, validate on 77 samples
Epoch 1/1
 - 1012s - loss: 1.2636 - acc: 0.5848 - val_loss: 1.1717 - val_acc: 0.5584


C:\Users\Annie\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


F1-score: 60.076888
1
Train on 8530 samples, validate on 77 samples
Epoch 1/1
 - 277s - loss: 0.4851 - acc: 0.8540 - val_loss: 1.0197 - val_acc: 0.5844
F1-score: 74.314783
2
Train on 8530 samples, validate on 77 samples
Epoch 1/1
 - 278s - loss: 0.2856 - acc: 0.9171 - val_loss: 0.9661 - val_acc: 0.6623
F1-score: 89.559083
3
Train on 8530 samples, validate on 77 samples
Epoch 1/1
 - 277s - loss: 0.2100 - acc: 0.9375 - val_loss: 0.9120 - val_acc: 0.6623
F1-score: 79.945183
4
Train on 8530 samples, validate on 77 samples
Epoch 1/1
 - 277s - loss: 0.1636 - acc: 0.9472 - val_loss: 0.9580 - val_acc: 0.7662
F1-score: 81.075940
5
Train on 8530 samples, validate on 77 samples
Epoch 1/1
 - 283s - loss: 0.1267 - acc: 0.9611 - val_loss: 0.9516 - val_acc: 0.7273
F1-score: 92.691849
6
Train on 8530 samples, validate on 77 samples
Epoch 1/1
 - 276s - loss: 0.1104 - acc: 0.9660 - val_loss: 0.9791 - val_acc: 0.7273
F1-score: 91.488039
7
Train on 8530 samples, validate on 77 samples
Epoch 1/1
 - 276s - 

In [208]:
for i in range(20):
    print(i)
    history = model.fit([padded_docs_pr3_train, padded_docs_pr2_train, padded_docs_pr_train, padded_docs_train, padded_docs_po_train, padded_docs_po2_train, padded_docs_po3_train], classes_train, epochs = 1, verbose=2, validation_data=([padded_docs_pr3_test, padded_docs_pr2_test, padded_docs_pr_test, padded_docs_test, padded_docs_po_test, padded_docs_po2_test, padded_docs_po3_test], classes_test))
    predict = np.argmax(model.predict([padded_docs_pr3_val, padded_docs_pr2_val, padded_docs_pr_val, padded_docs_val, padded_docs_po_val, padded_docs_po2_val, padded_docs_po3_val]), axis=1)
    answer = np.argmax(classes_val, axis=1)
    print('F1-score: %f' % (f1_score(predict, answer, average="macro")*100))

0
Train on 8530 samples, validate on 77 samples
Epoch 1/1
 - 311s - loss: 1.2751 - acc: 0.5762 - val_loss: 1.0238 - val_acc: 0.6623
F1-score: 64.842660
1
Train on 8530 samples, validate on 77 samples
Epoch 1/1
 - 226s - loss: 0.5087 - acc: 0.8404 - val_loss: 0.8282 - val_acc: 0.7273
F1-score: 84.787898
2
Train on 8530 samples, validate on 77 samples
Epoch 1/1
 - 229s - loss: 0.3210 - acc: 0.9001 - val_loss: 0.8087 - val_acc: 0.7792
F1-score: 93.419437
3
Train on 8530 samples, validate on 77 samples
Epoch 1/1
 - 225s - loss: 0.2426 - acc: 0.9225 - val_loss: 0.8167 - val_acc: 0.7143
F1-score: 81.294952
4
Train on 8530 samples, validate on 77 samples
Epoch 1/1
 - 227s - loss: 0.1815 - acc: 0.9437 - val_loss: 0.9744 - val_acc: 0.7143
F1-score: 92.967773
5
Train on 8530 samples, validate on 77 samples
Epoch 1/1
 - 229s - loss: 0.1524 - acc: 0.9513 - val_loss: 0.9655 - val_acc: 0.6883
F1-score: 92.197793
6
Train on 8530 samples, validate on 77 samples
Epoch 1/1
 - 223s - loss: 0.1330 - acc: 

In [ ]:
for i in range(20):
    print(i)
    history = model.fit([padded_docs_pr2_train, padded_docs_pr_train, padded_docs_train, padded_docs_po_train, padded_docs_po2_train], classes_train, epochs = 1, verbose=2, validation_data=([padded_docs_pr2_test, padded_docs_pr_test, padded_docs_test, padded_docs_po_test, padded_docs_po2_test], classes_test))
    predict = np.argmax(model.predict([padded_docs_pr2_val, padded_docs_pr_val, padded_docs_val, padded_docs_po_val, padded_docs_po2_val]), axis=1)
    answer = np.argmax(classes_val, axis=1)
    print('F1-score: %f' % (f1_score(predict, answer, average="macro")*100))

In [ ]:
### from sklearn.metrics import f1_score
for i in range(20):
    print(i)
    history = model.fit([padded_docs_pr_train, padded_docs_train, padded_docs_po_train], classes_train, epochs = 1, verbose=2, validation_data=([padded_docs_pr_test, padded_docs_test, padded_docs_po_test], classes_test))
    predict = np.argmax(model.predict([padded_docs_pr_val, padded_docs_val, padded_docs_po_val]), axis=1)
    answer = np.argmax(classes_val, axis=1)
    print('F1-score: %f' % (f1_score(predict, answer, average="macro")*100))

In [ ]:
### from sklearn.metrics import f1_score
for i in range(20):
    print(i)
    history = model.fit(padded_docs_train, classes_train, epochs = 1, verbose=2, validation_data=(padded_docs_test, classes_test))
    predict = np.argmax(model.predict(padded_docs_val), axis=1)
    answer = np.argmax(classes_val, axis=1)
    print('F1-score: %f' % (f1_score(predict, answer, average="macro")*100))